<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/master/train2vec%2Btag2vec%2Bmf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [2]:
pip install implicit

     |████████████████████████████████| 1.1MB 2.3MB/s 
  Created wheel for implicit: filename=implicit-0.4.2-cp36-cp36m-linux_x86_64.whl size=3420119 sha256=3af6c7e966726b86b06122a190206894ca387444dd045a73cdab722cdd60a986
  Stored in directory: /root/.cache/pip/wheels/1b/48/b1/1aebe3acc3afb5589e72d3e7c3ffc3f637dc4721c1a974dff7
Successfully built implicit


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
import scipy
warnings.filterwarnings('ignore')
import implicit

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [63]:
folder = '/content/gdrive/My Drive/melon kakao'
filelist = os.chdir(folder)

train=pd.read_json('train.json')
test=pd.read_json('test.json')
test_hyunju=pd.read_json('train2vec_addition.json ')

In [64]:
test

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[],70107,,"[398985, 449403, 411543, 528044, 143048, 98020...",6,2012-09-29 01:57:26.000
1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
4,[],102395,,"[630683, 481582, 528550, 285114, 506667, 17922...",38,2018-07-11 16:43:32.000
...,...,...,...,...,...,...
10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000
10738,[여름],2605,,"[234554, 265033, 507260, 83092, 366757, 497097...",4,2015-06-06 09:52:01.000


In [65]:
test_hyunju

,id,predicted_tags
0,70107,"[황현, 프리스타일, Apink, 고등학생, 서지음, 오늘의추천가수, 슴스테, Mo..."
1,102395,"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep..."
2,42586,"[weeflow, kxan, Empire, 히팝, 제국, 센힙합, SMTM, nb2..."
3,13244,"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피..."
4,55399,"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso..."
...,...,...
4293,120081,"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW..."
4294,16489,"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트..."
4295,46470,"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ..."
4296,48788,"[가요리믹스, 김경호, 지아, 플라이투더스카이, MCTHEMAX, 노래방인기곡, 이..."


In [67]:
test_id = test['id'].tolist()
test_h_id = test_hyunju['id'].tolist()
len_1 = test.shape[0]
len_2 = test_hyunju.shape[0]

for i in range(len_1) :
  for j in range(len_2) :
    if test_id[i] == test_h_id[j]:
      test['tags'][i] = test_hyunju['predicted_tags'][j]

In [78]:
train_tag=train[['id', 'tags']]
test_tag=test[['id', 'tags']]

plylst_tag_map=pd.concat([train_tag, test_tag])

plylst_tag_map_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map.id.values, list(map(len, plylst_tag_map.tags))),
        np.concatenate(plylst_tag_map.tags.values)
    )
)

plylst_tag_map=pd.DataFrame(data=plylst_tag_map_unnest[0], columns=plylst_tag_map.columns)
plylst_tag_map['id']=plylst_tag_map['id'].astype('category')
plylst_tag_map['tags']=plylst_tag_map['tags'].astype('category')

del plylst_tag_map_unnest

In [83]:
plylst_tag_map['id']=plylst_tag_map['id'].astype('category')
plylst_tag_map['tags']=plylst_tag_map['tags'].astype('category')

tag_sparse=scipy.sparse.coo_matrix((np.ones(plylst_tag_map.shape[0]),
                                       (plylst_tag_map['tags'].cat.codes.copy(), 
                                        plylst_tag_map['id'].cat.codes.copy())))

In [115]:
model_tag=implicit.als.AlternatingLeastSquares(factors=30, regularization=0.005, dtype=np.float64, iterations=40, use_gpu=False)
model_tag.fit(tag_sparse)

In [116]:
repos_tag=dict(enumerate(plylst_tag_map['tags'].cat.categories))

filter1=test_hyunju['predicted_tags'].astype('str')=='[]'
test_hyunju=test_hyunju[~filter1]

plylst_tag_map_test=test_hyunju[['id', 'predicted_tags']]
plylst_tag_map_test.columns=['id', 'tags']

plylst_tag_map_test_unnest=np.dstack(
    (
        np.repeat(plylst_tag_map_test.id.values, list(map(len, plylst_tag_map_test.tags))),
        np.concatenate(plylst_tag_map_test.tags.values)
    )
)

plylst_tag_map_test=pd.DataFrame(data=plylst_tag_map_test_unnest[0], columns=plylst_tag_map.columns)

del(plylst_tag_map_test_unnest)

In [117]:
plylst_tag_map_test['id']=plylst_tag_map_test['id'].astype('category')
plylst_tag_map_test['tags']=plylst_tag_map_test['tags'].astype('category')
test_sparse=scipy.sparse.coo_matrix((np.ones(plylst_tag_map_test.shape[0]),
                                       (plylst_tag_map_test['id'].cat.codes.copy(), 
                                        plylst_tag_map_test['tags'].cat.codes.copy()))).tocsr()

In [118]:
result_tag=[]
for i in range(test_hyunju.shape[0]):
  B=model_tag.recommend(userid=i, user_items=test_sparse, N=10)
  result_B=[]
  for j in range(len(B)):
    B[j]=repos_tag[B[j][0]]
    result_B.insert(j, B[j])

  result_tag.insert(i, result_B)


In [121]:
test_hyunju['predicted_tags'][0].extend(result_tag[0])

In [122]:
test_hyunju['predicted_tags'][0][:10]

['황현',
 '프리스타일',
 'Apink',
 '고등학생',
 '서지음',
 '오늘의추천가수',
 '슴스테',
 'Monotree',
 '잼팩토리',
 '모노트리']

In [127]:
test_hyunju=test_hyunju.reset_index().drop('index', axis=1)

In [128]:
new_result=[]
for i in range(test_hyunju.shape[0]):
  test_hyunju['predicted_tags'][i].extend(result_tag[i])
  new_result.insert(i, test_hyunju['predicted_tags'][i][:10])

In [129]:
test_hyunju['new']=new_result

In [142]:
collabo_result=test_hyunju[['id', 'new']]
collabo_result.columns=['id', 'predicted_tags']

In [143]:
collabo_result.to_json('성환+현주+재현.json')

In [135]:
collabo_result.shape[0]

4118

In [137]:
test1=pd.read_json('성환오빠 채운거.json')

In [140]:
filter2=test1['predicted_tags'].astype('str')=='[]'
test_nodata=test1[filter2]

In [145]:
collabo_result=pd.concat([collabo_result, test_nodata])

In [150]:
collabo_result.columns=['id', 'predicted_tags']

In [152]:
collabo_result.reset_index(inplace=True)

In [156]:
collabo_result=collabo_result.drop('index', axis=1)

In [157]:
collabo_result.to_json('J_result3.json')